In [1]:
from __future__ import division
import cPickle as pickle
import gzip
import torch as th
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from network import CNN

In [2]:
partitions = ('training', 'validation', 'test')
data = pickle.load(gzip.open('mnist.pkl.gz'))
data = dict(zip(partitions, data))

In [3]:
cnn = CNN()
n_digits = 9
path = 'trained-cnn-%d' % n_digits
state_dict = th.load(path)
cnn.load_state_dict(state_dict)
cnn.cuda()

CNN (
  (_conv0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (_conv1): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (_linear0): Linear (784 -> 84)
  (_linear1): Linear (84 -> 10)
)

In [4]:
batch_size = 1024
data_loaders = {}
for key, value in data.items():
    value = map(th.from_numpy, value)
    dataset = TensorDataset(*value)
    data_loaders[key] = DataLoader(dataset, batch_size)

In [5]:
def n_matches(p, labels):
    _, p = th.max(p, 1)
    n = th.sum(p == labels)
    n = n.data[0]
    return n

In [6]:
for partition, data_loader in data_loaders.items():
    ns, nm = 0, 0
    for batch in data_loader:
        data, labels = batch
        data = data.view(-1, 1, 28, 28)
        data, labels = Variable(data.cuda()), Variable(labels.cuda())
        data = cnn(data)
        ns += data.size()[0]
        nm += n_matches(data, labels)
    ratio = nm / ns
    print '%s %3f' % (partition, ratio)

test 0.113500
training 0.113560
validation 0.106400
